In [2]:
# installing all dependencies required for the notebook
%pip install -r requirements.txt

  Using cached blis-0.4.1-cp37-cp37m-macosx_10_6_intel.macosx_10_9_intel.macosx_10_9_x86_64.macosx_10_10_intel.macosx_10_10_x86_64.whl (4.0 MB)
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached certifi-2020.11.8-py2.py3-none-any.whl (155 kB)
  Using cached cymem-2.0.4-cp37-cp37m-macosx_10_9_x86_64.whl (31 kB)
ERROR: Could not find a version that satisfies the requirement en-core-web-sm==2.3.1 (from -r requirements.txt (line 9)) (from versions: none)
ERROR: No matching distribution found for en-core-web-sm==2.3.1 (from -r requirements.txt (line 9))
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

#import data
data = pd.read_csv("data/train_preprocessed.csv")
data.head()

,id,keyword,location_data,text_data,target_data,drinking,courtney,@blazerfan,#speakingfromexperience,@cortezera,...,http://t.co/ixw2cutk1c,#oklahoma,http://t.co/s4sicmyrmh,airplane,95,aaarrrgghhh,https://t.co/cvkqigr1az,screams~,#silver,http://t.co/cubc0nq6fd
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#### spell check all words using pyspellchecker

In [3]:
%pip install pyspellchecker

from spellchecker import SpellChecker

Note: you may need to restart the kernel to use updated packages.


In [4]:
## initialize spellchecker and test on a few words
spell = SpellChecker(distance=1)

test = ['na', 'lief']
for word in test:
    # Get the one `most likely` answer
    print(spell.correction(word))

na
life


In [5]:
bag_of_words = data.columns[4:]

In [6]:
## for each word (one-hot encoded as column names) replace with spellcheck version
column_map = {} #dictionary to store map for corrected words

n_corrected = 0 #count numbe of corrections made
#iterate through all columns that represent tweet words 
for word in bag_of_words:
    #filter out hashtags, mentions, and urls
    if '#' not in word and 'http://' not in word and '@' not in word:
        #use spell check to get correct version of word
        corrected = spell.correction(word)
        #if word is switched by spellchecker:
        if word != corrected:
            #store correct version in dictionary and print 
            column_map[word] = corrected
            n_corrected += 1
            print(word, 'switched to ', corrected)

itched to  caddie
lurkin switched to  lurking
prolly switched to  polly
/server switched to  server
sif- switched to  sift
cantar switched to  canter
3la switched to  la
mthe switched to  the
miì switched to  mix
westminister switched to  westminster
nanke switched to  nance
dvds switched to  dads
iphoto switched to  photo
orchs switched to  orcs
mygc switched to  myc
-legion switched to  legion
muzzie switched to  muzzle
polit switched to  polite
dolla switched to  dollar
xela switched to  hela
radler switched to  sadler
leitchfield switched to  litchfield
~27 switched to  a27
friggin switched to  frigging
stuff- switched to  stuff
amico switched to  amino
westeros switched to  westerns
yuuko switched to  yuko
trey switched to  they
doco switched to  dock
bouta switched to  bout
firefigther switched to  firefighter
mgs2 switched to  mgs
|lauren switched to  lauren
dlh switched to  doh
rockin switched to  rocking
bardo switched to  bard
minaj switched to  mina
x'mas switched to  xmas
k

In [7]:
print('% corrected: ', n_corrected / len(bag_of_words))

#rename columns to corrected version
data.rename(columns=column_map, inplace=True)

% corrected:  0.07122426970506869


In [ ]:
##write spellchecked to csv

In [11]:
##### cleaning locations

In [23]:
## use gazetteers corpus which includes: country names, us states and abbreviations, major us cities
#unfortunately this means that it might miss many non-us locations ~ something to improve on later
%pip install nltk
%pip install fuzzywuzzy

from nltk.corpus import gazetteers 
import numpy as np 
from fuzzywuzzy import process

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [13]:
#store all locations 
locations = set(gazetteers.words()) 

n_loc_cleaned = 0
for i, row in data.iterrows():
    clean_location = np.nan
    location = row['location_data']
    #filter out empty
    if type(location) == type(' '):
        #use fuzzywuzzy to find the best match in the locations library
        match = process.extractOne(location, locations)
        score = match[1] #fuzzywuzzy ratio score between the two strings
        match_location = match[0]
        #lower score for longer matches, to avoid over-matching abbreviations
        #otherwise it will match state abbreviations like 'nd' to longer words
        if score >= 90 and len(match_location) > 4:
            clean_location = match_location
            n_loc_cleaned += 1
        elif score >= 95:
            clean_location = match_location
            n_loc_cleaned += 1
    #store cleaned location in dataframe
    data.at[i, 'location_clean'] = clean_location

In [14]:
print('% locations cleaned: ', n_loc_cleaned / len(data['location_clean']))


% locations cleaned:  0.2799159332720347


In [15]:
## one-hot encode locations

#iterate through each location
unique_locations = list(data['location_clean'].unique())
for loc in unique_locations:
    if type(loc) == str:
        #insert column for one-hot encoding of this location; use try/except for running code multiple times
        #otherwise get error that you cant insert column twice
        try:
            data.insert(4, 'loc: ' + loc, [0 for i in range(len(data))])
        except:
            pass
        #encode all rows identified as being part of that location
        column = 'loc: ' + loc
        rows_in = data.loc[data['location_clean'] == loc]
        #encode each row
        for i in rows_in.index:
            data.at[i, column] = 1
        print(len(rows_in), len(data.loc[data[column] == 1]))

8 8
4 4
43 43
30 30
4 4
1 1
105 105
21 21
6 6
16 16
2 2
32 32
13 13
1 1
30 30
7 7
57 57
11 11
34 34
6 6
2 2
20 20
4 4
17 17
3 3
4 4
10 10
2 2
54 54
13 13
2 2
11 11
18 18
2 2
1 1
60 60
4 4
9 9
2 2
121 121
7 7
2 2
7 7
11 11
1 1
5 5
15 15
3 3
66 66
11 11
2 2
2 2
16 16
7 7
14 14
1 1
3 3
1 1
10 10
3 3
4 4
7 7
14 14
3 3
10 10
2 2
4 4
11 11
3 3
31 31
1 1
6 6
5 5
4 4
56 56
8 8
43 43
14 14
15 15
12 12
7 7
21 21
25 25
15 15
1 1
26 26
1 1
11 11
18 18
14 14
9 9
3 3
5 5
3 3
2 2
5 5
1 1
26 26
2 2
4 4
2 2
1 1
43 43
1 1
2 2
1 1
7 7
9 9
3 3
2 2
14 14
8 8
16 16
10 10
8 8
12 12
1 1
4 4
3 3
8 8
8 8
1 1
3 3
5 5
4 4
4 4
1 1
4 4
2 2
9 9
3 3
2 2
1 1
3 3
1 1
3 3
1 1
6 6
4 4
1 1
3 3
4 4
6 6
10 10
1 1
1 1
14 14
4 4
18 18
2 2
2 2
1 1
2 2
3 3
1 1
5 5
6 6
2 2
14 14
15 15
2 2
7 7
4 4
2 2
1 1
2 2
6 6
7 7
3 3
8 8
3 3
3 3
1 1
2 2
1 1
3 3
4 4
1 1
1 1
2 2
3 3
6 6
4 4
2 2
7 7
1 1
1 1
6 6
1 1
1 1
4 4
1 1
1 1
1 1
1 1
2 2
2 2
1 1
3 3
1 1
1 1
3 3
3 3
7 7
1 1
2 2
3 3
1 1
8 8
1 1
1 1
1 1
3 3
1 1
3 3
2 2
1 1
1 1
1 1
3 3
5 5
2 2


In [16]:
data.to_csv('spell_loc_train_preprocessed.csv')